In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
import pandas as pd
from sklearn.cluster import KMeans
from datetime import datetime
from tqdm import tqdm

In [2]:
FILE_PATH = "Dataset\EURUSD\EURUSD_M30_with_features.csv"

<>:1: SyntaxWarning: invalid escape sequence '\E'
<>:1: SyntaxWarning: invalid escape sequence '\E'
C:\Users\Stephen\AppData\Local\Temp\ipykernel_21860\891419644.py:1: SyntaxWarning: invalid escape sequence '\E'
  FILE_PATH = "Dataset\EURUSD\EURUSD_M30_with_features.csv"


In [3]:
# Load the dataset with features
data = pd.read_csv(FILE_PATH)

# Ensure DateTime is parsed correctly
data['DateTime'] = pd.to_datetime(data['DateTime'])

In [4]:
# Select features for training (e.g., SMA100, RSI14, Close)
features = ['SMA100', 'RSI14', 'Close']
X = data[features].values

# Normalize the features
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Convert to PyTorch tensors
X_tensor = torch.tensor(X_scaled, dtype=torch.float32).to('cuda')

In [5]:
# Define the Transformer model
class ForexTransformer(nn.Module):
    def __init__(self, input_dim, seq_length, num_heads, num_layers, output_dim):
        super(ForexTransformer, self).__init__()
        self.embed_dim = 128  # Ensure this is divisible by num_heads
        self.embedding = nn.Linear(input_dim, self.embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=self.embed_dim, nhead=num_heads)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(self.embed_dim * seq_length, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        x = self.transformer(x)
        x = x.flatten(start_dim=1)  # Flatten for the fully connected layer
        x = self.fc(x)
        return x


In [6]:
# Parameters
seq_length = 10  # Number of time steps
num_heads = 4
num_layers = 2
output_dim = 4  # Buy, Sell, Hold, Nothing

In [ ]:
# Prepare sequential data
X_seq = []
for i in range(len(X_scaled) - seq_length + 1):
    X_seq.append(X_scaled[i:i + seq_length])
X_seq = torch.tensor(X_seq, dtype=torch.float32).to('cuda')
# Generate pseudo-labels (unsupervised clustering approach)
def generate_pseudo_labels(X_scaled, n_clusters=4):
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    pseudo_labels = kmeans.fit_predict(X_scaled)
    pseudo_labels = pseudo_labels[:len(X_scaled) - seq_length + 1]
    return torch.tensor(pseudo_labels, dtype=torch.long).to('cuda')

pseudo_labels = generate_pseudo_labels(X_scaled)
# Initialize model, loss, and optimizer
model = ForexTransformer(input_dim=X_tensor.shape[1], seq_length=seq_length,  num_heads=num_heads, num_layers=num_layers, output_dim=output_dim).to('cuda')
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.00001)
                        
MODEL_PATH = "D:/Programing/AI Trader/Model/transformer_model_seqlength 14_20241215_125028.pth"
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()
print(f"\nModel: {MODEL_PATH.split('/')[-1]} is loaded.\n")
    
# Training loop
def train_transformer(model, X_seq, pseudo_labels, epochs=100, batch_size=4096):
    dataset = torch.utils.data.TensorDataset(X_seq, pseudo_labels)
    dataloader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True)
    for epoch in tqdm(range(epochs), desc="Training Epochs"):
        model.train()
        epoch_loss = 0
        for batch_X, batch_y in dataloader:
            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")
        print(f"Epoch {epoch+1}/{epochs}, Loss: {epoch_loss:.4f} ; {timestamp}")
        MODEL_SAVEPATH = f"./Model/transformerModel_seqlength {seq_length}_Loss {epoch_loss:.4f}_at {timestamp}.pth"
        
        # print(MODEL_SAVEPATH)
        
        torch.save(model.state_dict(), MODEL_SAVEPATH)

train_transformer(model, X_seq, pseudo_labels)

C:\Users\Stephen\AppData\Local\Temp\ipykernel_21860\631652508.py:5: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  X_seq = torch.tensor(X_seq, dtype=torch.float32).to('cuda')
c:\Program Files\Python312\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(
C:\Users\Stephen\AppData\Local\Temp\ipykernel_21860\631652508.py:20: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data w


Model: transformer_model_seqlength 14_20241215_125028.pth is loaded.



Training Epochs:   1%|          | 1/100 [01:04<1:45:46, 64.11s/it]

Epoch 1/100, Loss: 0.1917 ; 20241215-135151


Training Epochs:   2%|▏         | 2/100 [02:07<1:44:20, 63.89s/it]

...

Epoch 7/100, Loss: 0.1714 ; 20241215-135818


Training Epochs:   8%|▊         | 8/100 [08:35<1:39:23, 64.83s/it]

Epoch 8/100, Loss: 0.1709 ; 20241215-135923


Training Epochs:   9%|▉         | 9/100 [09:40<1:38:12, 64.75s/it]

Epoch 9/100, Loss: 0.1644 ; 20241215-140028


In [8]:
# Save the trained model
# MODEL_PATH = f"./Model/transformer_{seq_length}_model.pth"
# torch.save(model.state_dict(), MODEL_PATH)
# print(f"Model saved to {MODEL_PATH}")


In [9]:
# Predict target price and stop loss for buy/sell conditions
def calculate_target_stop_prices(close_price, condition):
    if condition == 0:  # Buy
        target_price = close_price * 1.02  # 2% increase
        stop_loss_price = close_price * 0.98  # 2% decrease
    elif condition == 1:  # Sell
        target_price = close_price * 0.98  # 2% decrease
        stop_loss_price = close_price * 1.02  # 2% increase
    else:  # Hold or Nothing
        target_price = stop_loss_price = close_price
    return target_price, stop_loss_price


In [ ]:
# Example usage
example_data = X_seq[:10]
model.eval()
with torch.no_grad():
    predictions = model(example_data)
conditions = torch.argmax(predictions, axis=1).cpu().numpy()
for i, condition in enumerate(conditions):
    close_price = data.iloc[i]['Close']
    target, stop_loss = calculate_target_stop_prices(close_price, condition)
    print(f"Condition: {condition}, Target Price: {target:.4f}, Stop Loss Price: {stop_loss:.4f}")
